In [2]:
pip install pywin32

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pywinauto

Note: you may need to restart the kernel to use updated packages.


In [288]:
import time, win32con, win32api, win32gui, ctypes, win32com.client
from pywinauto import clipboard
from pywinauto import application
from pywinauto import findwindows
import os

In [280]:
w = win32con

PBYTE256 = ctypes.c_ubyte * 256
_user32 = ctypes.WinDLL("user32")
PUL = ctypes.POINTER(ctypes.c_ulong)

GetKeyboardState = _user32.GetKeyboardState
SetKeyboardState = _user32.SetKeyboardState

PostMessage = win32gui.PostMessage
SendMessage = win32gui.SendMessage
FindWindow = win32gui.FindWindow
IsWindow = win32gui.IsWindow

GetCurrentThreadId = win32api.GetCurrentThreadId
MakeLong = win32api.MAKELONG

GetWindowThreadProcessId = _user32.GetWindowThreadProcessId
AttachThreadInput = _user32.AttachThreadInput
MapVirtualKeyA = _user32.MapVirtualKeyA
MapVirtualKeyW = _user32.MapVirtualKeyW

#키 코드
Enter = 0x0D
EndKey = 0x23
F10 = 0x79
Tab = 0x09

In [227]:
# 조합키
def PostKeyEx(hwnd, key, shift, specialkey):
    if IsWindow(hwnd):

        ThreadId = GetWindowThreadProcessId(hwnd, None)
        # 아스키 코드값으로 바꿔주는거
        lparam = MakeLong(0, MapVirtualKeyA(key, 0))
        msg_down = w.WM_KEYDOWN
        msg_up = w.WM_KEYUP

        if specialkey:
            lparam = lparam | 0x1000000

        if len(shift) > 0:
            pKeyBuffers = PBYTE256()
            pKeyBuffers_old = PBYTE256()

            SendMessage(hwnd, w.WM_ACTIVATE, w.WA_ACTIVE, 0)
            AttachThreadInput(GetCurrentThreadId(), ThreadId, True)
            GetKeyboardState(ctypes.byref(pKeyBuffers_old))

            for modkey in shift:
                if modkey == w.VK_MENU:
                    lparam = lparam | 0x20000000
                    msg_down = w.WM_SYSKEYDOWN
                    msg_up = w.WM_SYSKEYUP
                pKeyBuffers[modkey] |= 128

            SetKeyboardState(ctypes.byref(pKeyBuffers))
            time.sleep(0.01)
            PostMessage(hwnd, msg_down, key, lparam)
            time.sleep(0.01)
            PostMessage(hwnd, msg_up, key, lparam | 0xC0000000)
            time.sleep(0.01)
            SetKeyboardState(ctypes.byref(pKeyBuffers_old))
            time.sleep(0.01)
            AttachThreadInput(GetCurrentThreadId(), ThreadId, False)

        else:
            SendMessage(hwnd, msg_down, key, lparam)
            SendMessage(hwnd, msg_up, key, lparam | 0xC0000000)

In [228]:
#텍스트 입력 및 엔터
def SendReturn(hwnd):
    win32api.PostMessage(hwnd, win32con.WM_KEYDOWN, win32con.VK_RETURN, 0)
    time.sleep(0.01)
    win32api.PostMessage(hwnd, win32con.WM_KEYUP, win32con.VK_RETURN, 0)

In [229]:
#키 입력 관련 클래스들#
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

    
class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

    
class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]
    
    
class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                ("mi", MouseInput),
                ("hi", HardwareInput)]

    
    
class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]
#key 입력 관련 클래스들#


#키 누르기
def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput(hexKeyCode, 0, 0, 0, ctypes.pointer(extra))
    x = Input(ctypes.c_ulong(1), ii_)
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))


#키 떼기
def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput(hexKeyCode, 0, 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

In [230]:
# # 채팅방 열기
def open_chatroom(chatroom_name):
    # # 채팅방 목록 검색하는 Edit (채팅방이 열려있지 않아도 전송 가능하기 위하여)
    hwndkakao = win32gui.FindWindow(None, "카카오톡") #카카오톡이라는 이름으로 켜져있는 창 선택
    hwndkakao_edit1 = win32gui.FindWindowEx( hwndkakao, None, "EVA_ChildWindow", None)
    #위에서 선택한 카카오톡 안에 EVA_ChildWindow 속성을 가진 창 선택
    #EVA_ChildWindow와 같은 값은 visual stuido의 spy++ 같은 tool을 사용해서 확인 가능
    hwndkakao_edit2_1 = win32gui.FindWindowEx( hwndkakao_edit1, None, "EVA_Window", None)
    hwndkakao_edit2_2 = win32gui.FindWindowEx( hwndkakao_edit1, hwndkakao_edit2_1, "EVA_Window", None)
    #EVA_Window처럼 이름이 같은 속성이 여러 개 있는 경우 한 번 더 호출하면 다음 EVA_Window 선택
    hwndkakao_edit3 = win32gui.FindWindowEx( hwndkakao_edit2_2, None, "Edit", None)
    #Edit는 채팅방 검색창

    # # Edit에 검색 _ 입력되어있는 텍스트가 있어도 덮어쓰기됨
    win32api.SendMessage(hwndkakao_edit3, win32con.WM_SETTEXT, 0, chatroom_name)
    #chatroom_name을 채팅방 검색창에 입력
    time.sleep(1)   # 안정성 위해 필요
    SendReturn(hwndkakao_edit3) #엔터와 같은 역할, 채팅방이 켜짐
    time.sleep(1)

In [231]:
# # 채팅방 내용 내보내기
def kakao_export(chatroom_name):
    # # 핸들 _ 채팅방
    
    count = 0
    hwndMain = 0
    while(hwndMain == 0): #채팅방 열려있는지 확인
        time.sleep(0.1)
        hwndMain = win32gui.FindWindow( None, chatroom_name)
        count += 1
        if(count > 1000): #탈출 조건
            print("can not find chat room")
            return
    
    time.sleep(1)
    count = 0
    saveMain = 0
    while(saveMain == 0): #다른 이름으로 저장 창 열려있는지 확인
        time.sleep(0.1)
        PostKeyEx(hwndMain, ord('S'), [w.VK_CONTROL], False) #내보내기 단축키
        #PostKeyEx 조합 단축키 누르는 함수, ctrl + s는 채팅방 내보내기 단축키
        time.sleep(0.1)
        saveMain = win32gui.FindWindow( None, "다른 이름으로 저장")
        count += 1
        if(count > 1000): #탈출 조건
            print("can not find save window")
            return

    time.sleep(1)
    saveControl1 = win32gui.FindWindowEx(saveMain, None, "DUIViewWndClassName", None)
    saveControl2 = win32gui.FindWindowEx(saveControl1, None, "DirectUIHWND", None)
    saveControl3 = win32gui.FindWindowEx(saveControl2, None, "FloatNotifySink", None)
    saveControl4 = win32gui.FindWindowEx(saveControl3, None, "ComboBox", None)
    saveControl5 = win32gui.FindWindowEx(saveControl4, None, "Edit", None)
    saveButton = win32gui.FindWindowEx(saveMain, None, "Button", None) #저장 버튼 선택
    time.sleep(1)
    
    PostKeyEx(saveControl5, ord('C'), [w.VK_CONTROL], False) #기본 이름 복사, 기존 데이터와 이름이 겹치는 것 방지를 위해
    time.sleep(1)
    win32api.SendMessage(saveControl5, win32con.WM_SETTEXT, 0, "test#1234#") #임시, 채팅방 이름과 핀 번호 입력
    time.sleep(1)
    PressKey(EndKey) #키 누르기
    ReleaseKey(EndKey) #키 떼기, 커서 맨 끝으로 이동
    time.sleep(1)
    PostKeyEx(saveControl5, ord('V'), [w.VK_CONTROL], False) #복사한 기본 이름 붙여넣기
    
    win32api.SendMessage(saveButton, win32con.BM_CLICK, 0, 0) #위에서 선택한 저장 버튼 클릭
    time.sleep(1)
    win32api.SendMessage(hwndMain, win32con.WM_CLOSE, 0, 0) #열어놓은 채팅창 닫기

In [232]:
# # 채팅방 최대화로 맨 앞에 띄우기
def show_chatroom(chatroom_name):
    # # 채팅방 목록 검색하는 Edit (채팅방이 열려있지 않아도 전송 가능하기 위하여)
    hwndkakao = win32gui.FindWindow(None, "카카오톡")
    hwndkakao_edit1 = win32gui.FindWindowEx( hwndkakao, None, "EVA_ChildWindow", None)
    hwndkakao_edit2_1 = win32gui.FindWindowEx( hwndkakao_edit1, None, "EVA_Window", None)
    hwndkakao_edit2_2 = win32gui.FindWindowEx( hwndkakao_edit1, hwndkakao_edit2_1, "EVA_Window", None)
    hwndkakao_edit3 = win32gui.FindWindowEx( hwndkakao_edit2_2, None, "Edit", None)

    # # Edit에 검색 _ 입력되어있는 텍스트가 있어도 덮어쓰기됨
    win32api.SendMessage(hwndkakao_edit3, win32con.WM_SETTEXT, 0, chatroom_name)
    time.sleep(1)   # 안정성 위해 필요

    win32gui.ShowWindow(hwndkakao, win32con.SW_SHOWMAXIMIZED)
    shell = win32com.client.Dispatch("WScript.Shell")
    shell.SendKeys('%')
    win32gui.SetForegroundWindow(hwndkakao)
    
    time.sleep(1)
    
    PostKeyEx(hwndkakao, F10, [w.VK_SHIFT], False)

In [298]:
def test(chatroom_name): #최초로 사용하는 함수. 채팅방이름에 핀 번호 붙이기 단체톡 전용
    # # 채팅방 목록 검색하는 Edit (채팅방이 열려있지 않아도 전송 가능하기 위하여)
    hwndkakao = win32gui.FindWindow(None, "카카오톡")
    hwndkakao_edit1 = win32gui.FindWindowEx( hwndkakao, None, "EVA_ChildWindow", None)
    hwndkakao_edit2_1 = win32gui.FindWindowEx( hwndkakao_edit1, None, "EVA_Window", None)
    hwndkakao_edit2_2 = win32gui.FindWindowEx( hwndkakao_edit1, hwndkakao_edit2_1, "EVA_Window", None)
    hwndkakao_edit3 = win32gui.FindWindowEx( hwndkakao_edit2_2, None, "Edit", None)

    # # Edit에 검색 _ 입력되어있는 텍스트가 있어도 덮어쓰기됨
    win32api.SendMessage(hwndkakao_edit3, win32con.WM_SETTEXT, 0, chatroom_name)
    time.sleep(1)   # 안정성 위해 필요
    SendReturn(hwndkakao_edit3)
    time.sleep(1)
    #여기까지는 채팅방 열기와 동일한 기능
    
    hwndMain = win32gui.FindWindow( None, chatroom_name)
    #연 채팅방 선택
    
    time.sleep(5) #안정성 위해
    PostKeyEx(hwndMain, ord('B'), [w.VK_CONTROL], False) #채팅방 설정을 보기 위한 단축키 ctrl + b
    time.sleep(1)
    tttt = win32gui.FindWindow("#32770", None) #채팅방 설정 창 번호: 32770
    t1 = win32gui.FindWindowEx( tttt, None, "EVA_ChildWindow", None)
    t2 = win32gui.FindWindowEx( t1, None, "Edit", None) #채팅방 이름이 적혀있는 창
    #TO-DO
    win32api.SendMessage(t2, win32con.WM_SETTEXT, 0, "test") #임시, 설정한 값 채팅방 이름란에 입력
    time.sleep(1)
    PressKey(Tab)
    ReleaseKey(Tab)
    time.sleep(1)
    PressKey(Tab)
    ReleaseKey(Tab)
    time.sleep(1)
    PressKey(Tab)
    ReleaseKey(Tab)
    time.sleep(1)
    PressKey(Tab)
    ReleaseKey(Tab)
    time.sleep(1)
    PressKey(Enter)
    ReleaseKey(Enter)
    time.sleep(1) #채팅방 이름 적용을 위한 과정, tab 5번을 통해 커서를 저장으로 이동시키고, enter 통해서 저장 버튼 클릭
    
    win32api.SendMessage(hwndkakao, win32con.WM_CLOSE, 0, 0) #채팅창 닫기

In [312]:
def delete_txt():
    path = "C:/Users/USER/Desktop" #기본적으로 채팅방 내보내기하면 바탕화면에 저장됨
    file_list = os.listdir(path) #바탕화면에 위치한 파일 이름들 목록으로 받아옴
    print ("file_list: {}".format(file_list)) #파일 목록 확인용
    
    for file in file_list: #파일 목록들 접근해서 파일 마지막이 txt로 끝나면 삭제
        if(file[-3:] == 'txt'):
            win32api.DeleteFile(path + '/' + file)
    
    file_list = os.listdir(path)
    print ("file_list: {}".format(file_list)) #파일 목록 확인용(잘 삭제되었는지)

In [307]:
chatroom_list = [] #채팅방 이름을 저장할 배열
f = open("./chatroom_list.txt", 'r', encoding='UTF8') #채팅방 이름이 적혀있는 텍스트 파일 열기
for line in f:
    chatroom_list.append(line.strip()) #텍스트 파일 읽어서 chatroom_list에 채팅방 이름들 저장

In [308]:
for chatroom in chatroom_list: #chatroom_list에 저장된 채팅방 목록들에 접근
   open_chatroom(chatroom) #채팅방 열기
   kakao_export(chatroom) #채팅방 내보내기

In [309]:
for chatroom in chatroom_list:
    test(chatroom) #채팅방 이름 변경

In [313]:
delete_txt() #이전 채팅방 내보내기 파일들 삭제 #채팅방 내보내기 하기 전에 사용될 함수

file_list: ['desktop.ini']
file_list: ['desktop.ini']


In [ ]:
#핀번호 올리고 DB 핀번호나 채팅방 이름 등록